In [1]:
import tensorflow as tf
tf.set_random_seed(777)
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 색온도 데이터 가져오기
# cct 배열에 저장

import_data = np.loadtxt('./v5data/train_v5_data.csv', delimiter=',')
import_cct = import_data[:, 0]

cct = []

for i in range(88):
    cct.append(import_cct[i*772:(i+1)*772])
    
print(np.shape(cct))

(88, 772)


In [3]:
# 분별 색온도의 변화율 계산
# delta_cct에 저장

delta_cct = []

print(len(cct))
print(len(cct[0]))

for i in range(len(cct)):
    temp = []
    for j in range(771):
        temp.append((int)(cct[i][j+1] - cct[i][j]))
    delta_cct.append(temp)
    
usedata = []
scaler = MinMaxScaler(feature_range=(0, 10))
    
for i in range(len(delta_cct)):
    temp = []
    for j in range(5):
        temp.append(abs(delta_cct[i][154*j]))
    usedata.append(temp)
    
train_data = scaler.fit_transform(usedata)

print(train_data[1])

88
772
[0.04071554 0.14450867 0.04436557 0.0277585  0.06257822]


In [4]:
# 학습시킬 색온도 데이터 분별하여 라벨링
# x_data = 색온도 변화율 데이터
# y_data = 흐린날, 맑은날 (0, 1)
# 맑은날 8, 흐린날 8

#

x_data = [train_data[8], train_data[19], train_data[42], train_data[54], train_data[57], train_data[66], train_data[78], train_data[83], train_data[11], train_data[16],
         train_data[17], train_data[18], train_data[21], train_data[22], train_data[24], train_data[26]]
y_data = [[1],[1],[1],[1],[1],[1],[1],[1],[0],[0],[0],[0],[0],[0],[0],[0]]

test_x_data = [train_data[0], train_data[1], train_data[2], train_data[3], train_data[4], train_data[5], train_data[6], train_data[7]]
test_y_data = [[1], [1], [0], [0], [1], [1], [0], [1]]

# x_data = [[-1, 4, -7, -5, -14], [-60, 13, 2, 18, -9], [-38, 14, 6, 10, -12], [-21, 692, 652, 1, -4]]
# y_data= [[1], [1],  [0], [0]]

# x_data = [train_data[8], train_data[19], train_data[17], train_data[18]]
# y_data = [[1], [1], [0], [0]]

X = tf.placeholder(tf.float32, shape=[None, 5])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([5, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

H = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(Y * tf.log(H) + (1 - Y) * tf.log(1 - H))

train = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(cost)

predicted = tf.cast(H > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X:x_data, Y:y_data})
        if step % 2000 == 0:
            print(step, cost_val)
            
#     h, c, a = sess.run([H, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    h, c, a = sess.run([H,predicted, accuracy], feed_dict={X:test_x_data, Y:test_y_data})
    print("\nHypothesis (sigmoid output)\n ", h, "\n\ntest_y_data", test_y_data, "\n\nCorrect (predicted data)\n ", c, "\n\nAccuracy (정확도) : ", a)
#     print(h, c, a)

0 0.61198956
2000 0.5193974
4000 0.50711024
6000 0.49887094
8000 0.49231666
10000 0.48677832

Hypothesis (sigmoid output)
  [[0.57583797]
 [0.6307144 ]
 [0.64098793]
 [0.4206288 ]
 [0.5030209 ]
 [0.6060734 ]
 [0.3955904 ]
 [0.615806  ]] 

test_y_data [[1], [1], [0], [0], [1], [1], [0], [1]] 

Correct (predicted data)
  [[1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]] 

Accuracy (정확도) :  0.875
